# A/B Testing to Determine an effective intervention to decrease early Udacity course cancellation

## Import necessary libraries

In [24]:
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm

## 1. Udacity's A/B testing course

Udacity published a great free course for A/B Testing, also known as split tests, which are online experiments used to test potential improvements to a website or mobile app. This Python notebook is a walkthrough solution of the final project.

Udacity's A/B Testing course is presented by Google and focuses on design and analysis of A/B tests. 

The course covers how to choose and characterize metrics to evaluate your experiments, how to design an experiment with enough statistical power and how to analyze the results and draw valid conclusions.

## 2. Experiment overview 

Experiment Name: "Free Trial" Screener.

It is conducted by Udacity, with the overall business goal of maximizing course completion by students.

## 2.1 Current conditions before change 
At the time of this experiment, Udacity courses currently have two options on the course overview page: "start free trial", and "access course materials".
If the student clicks "start free trial", they will be asked to enter their credit card information, and then they will be enrolled in a free trial for the paid version of the course. After 14 days, they will automatically be charged unless they cancel first.
If the student clicks "access course materials", they will be able to view the videos and take the quizzes for free, but they will not receive coaching support or a verified certificate, and they will not submit their final project for feedback.

## 2.2 Description of experimented change 
In the experiment, Udacity tested a change where if the student clicked "start free trial", they were asked how much time they had available to devote to the course.
If the student indicated 5 or more hours per week, they would be taken through the checkout process as usual. If they indicated fewer than 5 hours per week, a message would appear indicating that Udacity courses usually require a greater time commitment for successful completion, and suggesting that the student might like to access the course materials for free.

At this point, the student would have the option to continue enrolling in the free trial, or access the course materials for free instead.

## 2.3 Experiment hypothesis 
The hypothesis was that this might set clearer expectations for students upfront, thus reducing the number of frustrated students who left the free trial because they didn't have enough time—without significantly reducing the number of students to continue past the free trial and eventually complete the course. If this hypothesis held true, Udacity could improve the overall student experience and improve coaches' capacity to support students who are likely to complete the course.

## 2.4 Experiment details 
The unit of diversion is a cookie, although if the student enrolls in the free trial, they are tracked by user-id from that point forward. 

The same user-id cannot enroll in the free trial twice. For users that do not enroll, their user-id is not tracked in the experiment, even if they were signed in when they visited the course overview page.

## 3. Metric choice

We need two types of metrics for a successful experiment (or at least, a safe one); Invariate and evaluation metrics. 

Invariate metrics are used for "sanity checks", that is, to make sure our experiment (the way we presented a change to a part of the population, as well as the way we collected the data) is not inherently wrong. Basically, this means we pick metrics which we consider not to change (not to be affected) because of our experiment and later make sure these metrics don't change drastically between our control and experiment groups.

Evaluation metrics on the other hand, are the metrics in which we expect to see a change, and are relevant to the business goals we aim to achieve. For each metric we state a  Dmin  - which marks the minimum change which is practically significant to the business. For instance, stating that any increase in retention that is under 2%, even if statistically significant, is not practical to the business.


## 3.1 Invariate metrics - sanity checks

Dmin of each metric

- Number of Cookies in Course Overview Page (unique daily cookies on page - Ck): 3000 cookies 
- Number of Clicks on Free Trial Button (unique daily cookies who clicked - Cl): 240 clicks
- Free Trial button Click-Through-Probability (Cl/Ck): 0.01

## 3.2 Evaluation metrics - performance indicators

Dmin of each metric

- Gross Conversion (enrolled / Cl): 0.01
- Retention (paid / enrolled): 0.01
- Net Conversion (paid / Cl): 0.0075	 

## 4. Baseline values of metrics

Udacity gives the following rough estimates for these metrics (presumably collected from aggregates on daily traffic)

In [25]:
# Place estimators into a dictionary for ease of use later
baseline = {"Cookies":40000,"Clicks":3200,"Enrollments":660,"CTP":0.08,"GConversion":0.20625,
           "Retention":0.53,"NConversion":0.109313}

## 4.1 Estimating standard deviation 
Once we collected these estimates, we should estimate the standard deviation of a metric, this is computed for sample size calculations and confidence intervals for our results. The more variant a metric is, the harder it is to reach a significant result. Assuming a sample size of 5,000 cookies visiting the course overview page per day (as given in project's instructions) - we want to estimate a standard deviation, for the evaluation metrics only. The sample size we are considering should be smaller than the "population" we collected and small enough to have two groups with that size.

## 4.2 Scaling collected data 
For all the calculations to follow we need to scale our collected counts estimates of metrics with the sample size we specified for variance estimation. In this case, from 40000 unique cookies to visit the course overview page per day, to 5000.

In [26]:
# Scale The counts estimates
baseline["Cookies"] = 5000
baseline["Clicks"] = baseline["Clicks"]*(5000/40000)
baseline["Enrollments"] = baseline["Enrollments"]*(5000/40000)
baseline

{'Cookies': 5000,
 'Clicks': 400.0,
 'Enrollments': 82.5,
 'CTP': 0.08,
 'GConversion': 0.20625,
 'Retention': 0.53,
 'NConversion': 0.109313}

## 4.3 Estimating analytically 

In order to estimate variance analytically, we can assume metrics which are probabilities (p̂) are binomially distributed.

This assumption is only valid when the unit of diversion of the experiment is equal to the unit of analysis (the denominator of the metric formula).

In the cases when this is not valid, the actual variance might be different and it is recommended to estimate it empirically.
For each metric, we need to plug two variables into the formula:
p̂ - baseline probability of the event to occur
n - sample size

In [27]:
# Correspond the dmin and denominator given in 3.1 to the conversion rate
conversion_dict = {
    'GConversion':[0.01,'Clicks'], # The first corresponds to d_min, the second corresponds to the denominator
    'NConversion':[0.0075,'Clicks']
}

def cal_sd(conversion,d_min,denominator):
    '''
    Conversion
    d_min 
    denominator (denominator of conversion rate)
    '''
    R = {}
    R['d_min'] = d_min 
    R['p'] = baseline[conversion]
    R['n'] = baseline[denominator]
    R['sd'] = round(mt.sqrt((R["p"]*(1-R["p"]))/R["n"]),4)
    print('{} standard deviation:'.format(conversion),R['sd'])

conversions = ['GConversion','NConversion']
for conversion in conversions:
    d_min = conversion_dict[conversion][0]
    denominator = conversion_dict[conversion][1]
    cal_sd(conversion,d_min,denominator)

GConversion standard deviation: 0.0202
NConversion standard deviation: 0.0156


## 5. Experiment sizing

Given  α = 0.05  (significance level) and  β = 0.2  (power), we want to estimate how many total pageviews (cookies who viewed the course overview page) we need in the experiment. This amount will be divided into the two groups: control and experiment. This calculation can be done using an online calculator.

## 5.1 Get z-score critical value and standard deviations 

We can use python's scipy.stats.norm package to get all the required methods for normal distribution. 

The ppf method gives us access to the Percent Point Function (ppf) or Quantile Function, and besides it being the inverse of the Cummulative Distribution Function (cdf), this is the functions that will give back our required critical z-score.

In [28]:
# Caculate sd1 and sd2
def get_sds(p,d):
    sd1 = mt.sqrt(2*p*(1-p))
    sd2 = mt.sqrt(p*(1-p)+(p+d)*(1-(p+d)))
    x = [sd1,sd2]
    return x

# Caculate z-score
def get_z_score(alpha):
    return norm.ppf(alpha)

# Caculate sample size
def get_sampSize(sds,alpha,beta,d):
    n=pow((get_z_score(1-alpha/2)*sds[0]+get_z_score(1-beta)*sds[1]),2)/pow(d,2)
    return n

# Specify the baseline data p and d
GC = {}
GC["p"] = baseline["GConversion"]
GC["d"] = 0.01
NC = {}
NC["p"] = baseline["NConversion"]
NC["d"] = 0.0075

## 5.2 Calculate sample size per metric 

Gross conversion: enrolled / clicks ("free trial button")

In [30]:
GC["SampSize"] = round(get_sampSize(get_sds(GC["p"],GC["d"]), 0.05, 0.2, GC["d"]))
GC["SampSize"] = round(GC["SampSize"]/0.08*2)
GC["SampSize"]

645875.0

Net conversion: paid / clicks

In [32]:
NC["SampSize"] = round(get_sampSize(get_sds(NC["p"],NC["d"]),0.05,0.2,NC["d"]))
NC["SampSize"] = NC["SampSize"] / 0.08*2
NC["SampSize"]

685325.0

## 6. A/B testing experiment analysis 

Load data

In [34]:
control = pd.read_csv("/Users/miaaa/ab-testing/control_data.csv")
experiment = pd.read_csv("/Users/miaaa/ab-testing/experiment_data.csv")
control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


## 6.1 Sanity checks

First thing we have to do before even beginning to analyze this experiment's results is sanity checks. 

These checks help verify that the experiment was conducted as expected and that other factors did not influence the data which we collected.

This also makes sure that data collection was correct.

We have 3 Invariant metrics::

- Number of cookies in course overview page
- Number of clicks on free trial button
- Free trial button click-through-probability

Two of these metrics are simple counts like number of cookies or number of clicks and the third is a probability (CTP). We will use two different ways of checking whether these obsereved values are like we expect (if in fact the experiment was not damaged.

Sanity checks for differences between counts 
Number of cookies who viewed the course overview page - Starting from this simpel invariant metric, we want to count the total amount of cookie pageviews we diverted to each group and see if there is a significant difference int he amount of cookies. A significant difference will imply a biased experiment that we should not rely on it's results.

In [41]:
pageviews_cont = control['Pageviews'].sum()
pageviews_exp = experiment['Pageviews'].sum()
pageviews_total = pageviews_cont + pageviews_exp
print ("Number of pageviews in control:", pageviews_cont)
print ("Number of pageviews in experiment:", pageviews_exp)

Number of pageviews in control: 345543
Number of pageviews in experiment: 344660


These look like pretty close numbers. Now, let's make sure this difference in amounts is not significant and is random and even like we expected. We can model this diversion in the following way:
We expect the amount of pageviews in the control group to be about a half (50%) of the total pageviews in both groups, so we can define a random variable with an easy to use distribution.
A binomial random variable will be the number of successes we can expect to get out of N experiments, given the probability of a single success. 

So, if we consider being assigned to a group (control, for example) a success with probability 0.5 (random!), the number of samples which get assigned to the group is the value of our random binomial variable!

This get's easier thanks to the central limit theorem 

p = 0.5
alpha = 0.05
p_hat = round(pageviews_cont/(pageviews_total),4)
sd = mt.sqrt(p*(1-p)/(pageviews_total))
ME = round(get_z_score(1-(alpha/2))*sd,4)
print("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

The range of the confidence interval is between 0.4988 and 0.5012, and the sample value is 0.5006

As can be seen

The sample ratio of the control group is 0.5006 within the confidence interval, so the pv (page uv) indicator is passed the test, and there is no significant difference.

In [43]:
clicks_cont=control['Clicks'].sum()
clicks_exp=experiment['Clicks'].sum()
clicks_total=clicks_cont+clicks_exp

p_hat = round(clicks_cont/clicks_total,4)
sd = mt.sqrt(p*(1-p)/clicks_total)
ME = round(get_z_score(1-(alpha/2))*sd,4)
print("The confidence interval is between",p-ME,"and",p+ME,"; Is",p_hat,"inside this range?")

The confidence interval is between 0.4959 and 0.5041 ; Is 0.5005 inside this range?


The range of the error confidence interval is between 0.4959 and 0.5041, and the sample value is 0.5005.

We have another pass! Great, so far it still seems all is well with our experiment results. Now, for the final metric which is a probability.

## 6.2 Examining the effect size 

The next step is looking at the changes between the control and experiment groups with regard to our evaluation metrics to make sure the difference is there, that it is statistically significant and most importantly practically significant (the difference is "big" enough to make the experimented change beneficial to the company).

Now, all that is left is to measure for each evaluation metric, the difference between the values from both groups. Then, we compute the confidence interval for that difference and test whether or not this confidence interval is both statistically and practically significant.

- Gross Conversion: a metric is statistically significant if the confidence interval does not include 0 (that is, you can be confident there was a change), and it is practically significant if the confidence interval does not include the practical significance boundary (that is, you can be confident there is a change that matters to the business.)


Important: The given spreadsheet lists pageviews and clicks for 39 days, while it only lists enrollments and payments for 23 days. So, when working with enrollments and payments we should notice using only the corresponding pageviews and clicks, and not all of them

In [45]:
# Count the effective clicks of the experimental group and the control group
clicks_cont = control["Clicks"].loc[control["Enrollments"].notnull()].sum()
clicks_exp = experiment["Clicks"].loc[experiment["Enrollments"].notnull()].sum()

Gross Conversion:

In [50]:
#Gross Conversion - number of enrollments divided by number of clicks
enrollments_cont=control["Enrollments"].sum()
enrollments_exp=experiment["Enrollments"].sum()

GC_cont=enrollments_cont/clicks_cont
GC_exp=enrollments_exp/clicks_exp
GC_pooled=(enrollments_cont+enrollments_exp)/(clicks_cont+clicks_exp)
GC_sd_pooled=mt.sqrt(GC_pooled*(1-GC_pooled)*(1/clicks_cont+1/clicks_exp))
GC_ME=round(get_z_score(1-alpha/2)*GC_sd_pooled,4)
GC_diff=round(GC_exp-GC_cont,4)
print("The change due to the experiment is",GC_diff*100,"%")
print("Confidence Interval: [",GC_diff-GC_ME,",",GC_diff+GC_ME,"]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if",-GC["d_min"],"is not in the CI as well.")

The change due to the experiment is -2.06 %
Confidence Interval: [ -0.0292 , -0.012 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if -0.01 is not in the CI as well.


According to this result there was a change due to the experiment, that change was both statistically and practically significant. We have a negative change of 2.06%, when we were willing to accept any change greater than 1%. This means the Gross Conversion rate of the experiment group (the one exposed to the change, i.e. asked how many hours they can devote to studying) has decreased as expected by 2% and this change was significant. This means less people enrolled in the Free Trial after due to the pop-up.

Net Conversion: the hypothesis is the same as before just with net conversion instead of gross. At this point we expect the fraction of payers (out of the clicks) to decrease as well.

In [56]:
#Net Conversion - number of payments divided by number of clicks
payments_cont=control["Payments"].sum()
payments_exp=experiment["Payments"].sum()

NC_cont = payments_cont/clicks_cont
NC_exp = payments_exp/clicks_exp
NC_pooled = (payments_cont+payments_exp)/(clicks_cont+clicks_exp)
NC_sd_pooled = mt.sqrt(NC_pooled*(1-NC_pooled)*(1/clicks_cont+1/clicks_exp))
NC_ME = round(get_z_score(1-alpha/2)*NC_sd_pooled,4)
NC_diff = round(NC_exp-NC_cont,4)
print("The change due to the experiment is", NC_diff*100,"%")
print("Confidence Interval: [",NC_diff-NC_ME, ",", NC_diff+NC_ME, "]")
print ("The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if", 0.0075, "is not in the CI as well.")

The change due to the experiment is -0.49 %
Confidence Interval: [ -0.0116 , 0.0018000000000000004 ]
The change is statistically significant if the CI doesn't include 0. In that case, it is practically significant if 0.0075 is not in the CI as well.


Indicator changes due to strategy: -0.49%
Confidence interval: [-0.0116, 0.0018]

The confidence interval of the error does not include 0, and the D_min of NC is 0.75%, so the index difference is statistically significant, but it is not significant in practical significance, that is, the benefit is not significant.

## 7. Double check with sign tests

In a sign test we get another angle at analyzing the results we got - we check if the trend of change we observed (increase or decrease) was evident in the daily data. We are goint to compute the metric's value per day and then count on how many days the metric was lower in the experiment group and this will be the number of succssesses for our binomial variable. Once this is defined we can look at the proportion of days of success out of all the available days.

## 7.1 Data preparation 

In [57]:
# Let's first create the dataset we need for this:
# Start by merging the two datasets
full = control.join(other=experiment,how="inner",lsuffix="_cont",rsuffix="_exp")
# Let's look at what we got
full.count()

Date_cont           37
Pageviews_cont      37
Clicks_cont         37
Enrollments_cont    23
Payments_cont       23
Date_exp            37
Pageviews_exp       37
Clicks_exp          37
Enrollments_exp     23
Payments_exp        23
dtype: int64

In [58]:
# Now we only need the complete data records
full = full.loc[full["Enrollments_cont"].notnull()]
full.count()

Date_cont           23
Pageviews_cont      23
Clicks_cont         23
Enrollments_cont    23
Payments_cont       23
Date_exp            23
Pageviews_exp       23
Clicks_exp          23
Enrollments_exp     23
Payments_exp        23
dtype: int64

In [59]:
# Now, derive a new column for each metric, so we have it's daily values
# We need a 1 if the experiment value is greater than the control value=
x = full['Enrollments_cont']/full['Clicks_cont']
y = full['Enrollments_exp']/full['Clicks_exp']
full['GC'] = np.where(x<y,1,0)
# The same now for net conversion
z = full['Payments_cont']/full['Clicks_cont']
w = full['Payments_exp']/full['Clicks_exp']
full['NC'] = np.where(z<w,1,0)
full.head()

,Date_cont,Pageviews_cont,Clicks_cont,Enrollments_cont,Payments_cont,Date_exp,Pageviews_exp,Clicks_exp,Enrollments_exp,Payments_exp,GC,NC
0,"Sat, Oct 11",7723,687,134.0,70.0,"Sat, Oct 11",7716,686,105.0,34.0,0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,"Sun, Oct 12",9288,785,116.0,91.0,0,1
2,"Mon, Oct 13",10511,909,167.0,95.0,"Mon, Oct 13",10480,884,145.0,79.0,0,0
3,"Tue, Oct 14",9871,836,156.0,105.0,"Tue, Oct 14",9867,827,138.0,92.0,0,0
4,"Wed, Oct 15",10014,837,163.0,64.0,"Wed, Oct 15",9793,832,140.0,94.0,0,1


In [61]:
GC_x = full.GC[full["GC"]==1].count()
NC_x = full.NC[full["NC"]==1].count()
n = full.NC.count()
print("No. of cases for GC:",GC_x,'\n',
      "No. of cases for NC:",NC_x,'\n',
      "No. of total cases",n)

No. of cases for GC: 4 
 No. of cases for NC: 10 
 No. of total cases 23


## 7.2 Building a Sign Test

We can forget all about this part and just use an online sign test calculator, but I still want to implement the calculations behind it.

In [63]:
# First a function for calculating probability of x=number of successes
def get_prob(x,n):
    p=round(mt.factorial(n)/(mt.factorial(x)*mt.factorial(n-x))*0.5**x*0.5**(n-x),4)
    return p
# Next a function to compute the pvalue from probabilities of maximum x
def get_2side_pvalue(x,n):
    p = 0
    for i in range(0,x+1):
        p = p+get_prob(i,n)
    return 2*p

Finally, to conduct the sign test itself: we will calculate the p-value for each metric, using the counts GC_x,NC_x and n and the function get_2side_pvalue.

In [64]:
print ("GC Change is significant if",get_2side_pvalue(GC_x,n),"is smaller than 0.05")
print ("NC Change is significant if",get_2side_pvalue(NC_x,n),"is smaller than 0.05")

GC Change is significant if 0.0026000000000000003 is smaller than 0.05
NC Change is significant if 0.6774 is smaller than 0.05


We get the same conclusions as we got from our effect size calculation: the change in Gross Conversion was indeed significant, while the change in Net Conversion was not.

## 8. Conclusion

At this point, once we have seen that the actual underlying goal we had was not reached (increase fraction of paying users by asking them in advance if they have the time to invest in the course), we can only recommend to not continue with change. It may have caused a change in Gross conversion, but it didn't for net conversion.